In [1]:
#######thumbnail Scrapper working fine


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.support import expected_conditions as EC
import re




def scroll_to_end(wd, len_to_scroll: int = 4000, Sleep_between_scroll=5):
    prev_h = 0
    while True:
        wd.execute_script(f"window.scrollTo({prev_h},{prev_h + 1500})")
        time.sleep(Sleep_between_scroll)
        prev_h += 1500
        if prev_h >= (len_to_scroll):
            break


def scrape(urlyour, your_driver,no_of_video: int=1, len_to_scroll=5000):
    your_driver.get(urlyour)
    your_driver.implicitly_wait(5)
    your_driver.execute_script("window.scrollTo(0,900);")
    your_driver.implicitly_wait(5)
    your_driver.find_element(By.CSS_SELECTOR,
                             '#tabsContent .ytd-c4-tabbed-header-renderer:nth-child(4) .tp-yt-paper-tab').click()
    your_driver.implicitly_wait(5)

    prev_h = 0
    while True:
        driver.execute_script(f"window.scrollTo({prev_h},{prev_h + 200})")
        time.sleep(2)

        title_box = driver.find_elements(By.XPATH, '//*[@id="video-title"]')

        new_list_title = []
        for i in title_box:
            k = i.get_attribute('title')
            new_list_title.append(k)
            if len(new_list_title) >= no_of_video:
                break

        view_box = your_driver.find_elements(By.XPATH, '//*[@id="metadata-line"]/span[1]')
        new_list_views = []
        for k in view_box:
            v = k.text
            new_list_views.append(v)
            if len(new_list_views) >= no_of_video:
                break

        time_box = your_driver.find_elements(By.XPATH, '//*[@id="metadata-line"]/span[2]')
        new_list_upload = []
        for l in time_box:
            upl = l.text
            new_list_upload.append(upl)
            if len(new_list_upload) >= no_of_video:
                break

        url_box = your_driver.find_elements(By.CSS_SELECTOR, '#video-title')
        new_list_url = []
        for j in url_box:
            u = j.get_attribute('href')
            new_list_url.append(u)
            if len(new_list_url) >= no_of_video:
                break

        if len(set(new_list_url)) >= no_of_video:
            break

    my_data_youtube = pd.DataFrame(
        {'title': new_list_title, "url": new_list_url, "views": new_list_views, "upload": new_list_upload})
    return my_data_youtube

In [9]:
def Fetch_comments(driver, my_data_youtube, no_comments=5):
    
    list_all_data = []
    for url, title, views, upload in zip(my_data_youtube['url'], my_data_youtube['title'], my_data_youtube['views'],
                                         my_data_youtube['upload']):
        driver.get(url)
        driver.maximize_window()

        prev_h = 0
        comments = []
        commentors = []
        while True:
            driver.execute_script(f"window.scrollTo({prev_h},{prev_h + 200})")
            time.sleep(5)
            prev_h += 200
            comment = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
            name = driver.find_elements(By.XPATH, '//*[@id="author-text"]/span')

            for i in comment:
                try:
                    string = i.text
                    i = re.sub("\n", '', string)

                    if len(comments) < no_comments:
                        comments.append(i)
                except:
                    comments.append('none')

                    break

            for j in name:
                try:
                    s = j.text
                    if s == '':
                        s = s.replace('', "youtuber_comment")

                    if len(commentors) < no_comments:
                        commentors.append(s)
                except:
                    commentors.append(s)
                    break

            if len(comments) and len(commentors) >= no_comments:
                break
        comment_list = pd.DataFrame({'Comments':comments,"Commentors_Name" : commentors})
        comment_list['title'] = title
        comment_list['url'] = url
        comment_list['views'] = views
        comment_list['upload'] = upload
        list_all_data.append(comment_list)
    driver.close()
    return list_all_data 


In [10]:
S = Service("D:\software\chromedriver.exe")
driver = webdriver.Chrome(service=S)
k=scrape(urlyour='https://www.youtube.com/user/krishnaik06/featured', your_driver=driver, no_of_video=5)
df = Fetch_comments(driver, k, no_comments=25)

In [11]:
df1= pd.concat(df)

In [12]:
df1.to_csv('all_comment2.csv')